## A basic GAN using Keras

### Generator

In [24]:
import keras
from keras import layers
import numpy as np

In [25]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [26]:
generator_input = keras.Input(shape=(latent_dim))

# Transforms the input into a 16 * 16 * 128 channel feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsampling to 32 x 32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produces a 32x32 1-channel feature map (shape of a CIFAR 10 image)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
# Instantiate the generator model, which maps the input of shape(latent_dim) into an image of 32x32x3
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 32, 32, 256)       1048

### Discriminator

In [27]:
#from keras.optimizers import rmsprop_v2

discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# One dropout layer an important trick to introduce some randomness so that GAN will not stuck during training
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

# Instantiate the discriminator which turns a 32x32x3 input into a binary classification decision(real or fake)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

#Using Gradient clipping(by value) in optimizer
# To stabilize training uses learning rate decay
d_optimizer = keras.optimizers.rmsprop_v2.RMSprop(lr=0.008, clipvalue=1.0, decay=1e-8)

discriminator.compile(optimizer=d_optimizer, loss='binary_crossentropy')

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 6, 6, 128)         0  

### Adversial Network

In [28]:
# setting the weights to non trainable , this will only apply to the GAN model
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optmizer = keras.optimizers.rmsprop_v2.RMSprop(learning_rate=0.004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optmizer, loss='binary_crossentropy')

### Implementing GAN training

In [29]:
import os
from keras.preprocessing import image
from keras.datasets import cifar10
(x_train, y_train), (_ , _) = cifar10.load_data()

In [30]:
# Selecting only frog image as we will trin with onely class out of 10
x_train = x_train[y_train.flatten() == 6]

# Normalizing the data
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 100
batch_size = 20
save_dir_gen = './generated_image_folder'
save_dir_real = './real_image_folder'

# sample random points from latent space
start = 0
for step in range(iterations):
  random_latent_vector = np.random.normal(size=(batch_size, latent_dim))

  # Decode them to fake image
  generated_images = generator.predict(random_latent_vector)

  # Combine fake image with original image
  stop = start + batch_size
  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])

  # Assembling labels, discrminating between real and fake
  labels = np.concatenate([np.ones((batch_size,1)), np.zeros((batch_size,1))])
  # Add random noise to labels
  labels = labels + 0.05 * np.random.random(labels.shape)

  # Trains the discriminator
  d_loss = discriminator.train_on_batch(combined_images, labels)
  
  # samples the random points in the latent space
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

  # Assembling labels that says these are all real images(lie)
  misleading_targets = np.zeros((batch_size, 1))
  
  # Tains the generator via the GAN model where the discrminitaor weights are frozen
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

  start = start + batch_size
  if start > len(x_train) - batch_size:
    start = 0

  # Occasionaly saves and plots every 100 epochs
  if step % 100 == 0:
    gan.save_weights('gan.h5')

  # print metrics
  print(f"Discrminator loss: {d_loss}")
  print(f"Adversial loss: {a_loss}")

  # Save one generated image
  img = image.array_to_img(generated_images[0] * 255., scale=False)
  img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

  # Save one real image for comparison
  img = image.array_to_img(real_images[0] * 255, scale=False)
  img.save(os.path.join(save_dir_real, 'real_frog' + str(step) + '.png'))

Discrminator loss: 0.6876260042190552
Adversial loss: 5.019658276414702e-15
Discrminator loss: 34.291839599609375
Adversial loss: 0.0
Discrminator loss: 2672.966796875
Adversial loss: 8135.3095703125
Discrminator loss: 97008.28125
Adversial loss: 102641.6640625
Discrminator loss: -221.44308471679688
Adversial loss: 422450.0625
Discrminator loss: -4380.3193359375
Adversial loss: 1915486.625
Discrminator loss: -27179.90234375
Adversial loss: 6485943.0
Discrminator loss: -82714.234375
Adversial loss: 17741420.0
Discrminator loss: -183052.71875
Adversial loss: 39165248.0
Discrminator loss: -430937.5625
Adversial loss: 74572208.0
Discrminator loss: -834048.6875
Adversial loss: 133645648.0
Discrminator loss: -1729014.75
Adversial loss: 220806400.0
Discrminator loss: -2624334.0
Adversial loss: 348074016.0
Discrminator loss: -3350828.5
Adversial loss: 516779360.0
Discrminator loss: -5204601.0
Adversial loss: 729455744.0
Discrminator loss: -8776949.0
Adversial loss: 1001940288.0
Discrminator lo